In [119]:
import Bio
from Bio import Entrez 
from Bio import SeqIO
import urllib.request as ur

In [120]:
Entrez.email = "kellyhuang@berkeley.edu"
handle = Entrez.einfo() # or esearch, efetch, ...
record = Entrez.read(handle)
handle.close()

In [121]:
kegg_hsa = "http://rest.kegg.jp/link/hsa/hsa00010" #List of human genes in pathway hsa00010 (glycolysis)
kegg_eco = "http://rest.kegg.jp/link/eco/eco00010" #List of E. coli genes in pathway eco00010 (glycolysis)
kegg_dme = "http://rest.kegg.jp/link/dme/dme00010" #List of fruit fly genes in pathway dme00010 (glycolysis)
filehandler = ur.urlopen (kegg_hsa)
test = []
for line in filehandler:
    test.append(str(line).strip("b'"))
    line = str(line).replace("t","")
print(test)

['path:hsa00010\\thsa:10327\\n', 'path:hsa00010\\thsa:124\\n', 'path:hsa00010\\thsa:125\\n', 'path:hsa00010\\thsa:126\\n', 'path:hsa00010\\thsa:127\\n', 'path:hsa00010\\thsa:128\\n', 'path:hsa00010\\thsa:130\\n', 'path:hsa00010\\thsa:130589\\n', 'path:hsa00010\\thsa:131\\n', 'path:hsa00010\\thsa:160287\\n', 'path:hsa00010\\thsa:1737\\n', 'path:hsa00010\\thsa:1738\\n', 'path:hsa00010\\thsa:2023\\n', 'path:hsa00010\\thsa:2026\\n', 'path:hsa00010\\thsa:2027\\n', 'path:hsa00010\\thsa:217\\n', 'path:hsa00010\\thsa:218\\n', 'path:hsa00010\\thsa:219\\n', 'path:hsa00010\\thsa:220\\n', 'path:hsa00010\\thsa:2203\\n', 'path:hsa00010\\thsa:221\\n', 'path:hsa00010\\thsa:222\\n', 'path:hsa00010\\thsa:223\\n', 'path:hsa00010\\thsa:224\\n', 'path:hsa00010\\thsa:226\\n', 'path:hsa00010\\thsa:229\\n', 'path:hsa00010\\thsa:230\\n', 'path:hsa00010\\thsa:2538\\n', 'path:hsa00010\\thsa:2597\\n', 'path:hsa00010\\thsa:26330\\n', 'path:hsa00010\\thsa:2645\\n', 'path:hsa00010\\thsa:2821\\n', 'path:hsa00010\\ths

In [122]:
handle = Entrez.esummary(db="pubmed", id="19304878,14630660", retmode="xml")
records = Entrez.parse(handle)
for record in records:
    print(record['Title'])
handle.close()

Biopython: freely available Python tools for computational molecular biology and bioinformatics.
PDB file parser and structure class implemented in Python.


In [123]:
handle = Entrez.efetch(db="nucleotide", id="AY851612", rettype="gb", retmode="text")
print(handle.readline().strip())

LOCUS       AY851612                 892 bp    DNA     linear   PLN 10-APR-2007


In [124]:
handle = Entrez.esearch(db="nucleotide", term="homo sapiens[ORGN] BCRA1", sort="relevance", idtype="acc")
for i in Entrez.read(handle)["IdList"]:
    handle = Entrez.efetch(db="nucleotide", id =i, rettype ="fasta", retmode="text")
    print(handle.read())

>U11292.1 Homo sapiens Ki nuclear autoantigen mRNA, complete cds
GGGCGGACAGGCACAGAGGGAGGGAGCGAGCGAGCAGTGAGTAAGCCAGCAAGGGCGGTCGGGTCCCGAG
GTCAGCCGAGATTTCTCAGGTCCCTCCGGCCCCCTCCCTGGAGTCCACAGCGCCTCCGGTGTCCAGAGGA
TCGGACACGGCCCGGCCCGGCCATGGCCTCGTTGCTGAAGGTGGATCAGGAAGTGAAGCTCAAGGTTGAT
TCTTTCAGGGAGCGGATCACAAGTAAGGCAGAAGACTTGGTGGCAAATTTTTTCCCAAAGAAGTTATTAG
AACTTGATAGTTTTCTGAAGGAACCAATCTTAAACATCCATGACCTAACTCAGATCCACTCTGACATGAA
TCTCCCAGTCCCTGACCCCATTCTTCTCACCAATAGCCATGATGGACTGGATGGTCCCACTTATAAGAAG
CGAAGGTTGGATGAGTGTGAAGAAGCCTTCCAAGGAACCAAGGTGTTTGTGATGCCCAATGGGATGCTGA
AAAGCAACCAGCAGCTGGTGGACATTATTGAGAAAGTGAAACCTGAGATCCGGCTGTTGATTGAGAAATG
TAACACGCCTTCAGGCAAAGGTCCTCATATATGTTTTGACCTCCAGGTCAAAATGTGGGTACAGCTCCTG
ATTCCCAGGATAGAAGATGGAAACAACTTTGGGGTGTCCATTCAGGAGGAAACAGTTGCAGAGCTAAGAA
CTGTTGAGAGTGAAGCTGCATCTTATCTGGACCAGATTTCTAGATATTATATTACAAGAGCCAAATTGGT
TTCTAAAATAGCTAAATATCCCCATGTGGAGGACTATCGCCGCACCGTGACAGAGATTGATGAGAAAGAA
TATATCAGCCTTCGGCTCATCATATCAGAGCTGAGGAATCAATATGTCACTCTACATGACATGATCCTGA
AAAATATCGAGA

### Establish connection to database and create Genes table, Pathways table, and Enzyme table

In [125]:
import sqlite3
from sqlite3 import Error
 
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        conn.close()
 
if __name__ == '__main__':
    create_connection("/home/nathan/computational_biology_bioe131/lab4/sqldb.db")

2.6.0


In [126]:
conn = sqlite3.connect('sqldb.db')
c  = conn.cursor()

### Create Genes table, Pathways table, and Enzyme table

In [127]:
c.execute("""DROP TABLE genes""")
c.execute("""DROP TABLE pathways""")
c.execute("""DROP TABLE enzymes""")
c.execute("""DROP TABLE enzyme_pathway""")
# c.execute("""CREATE TABLE genes (id INT PRIMARY KEY ASC, name TEXT, description TEXT, organism TEXT, nucleotide_seq TEXT, chromosome TEXT, start INT, end INT, strand VARCHAR(1), translated_seq TEXT);""")
c.execute("""CREATE TABLE genes (name TEXT, description TEXT, chromosome TEXT, start TEXT, end TEXT);""")
c.execute("""CREATE TABLE pathways(name TEXT, description TEXT);""")
c.execute("""CREATE TABLE enzymes(name TEXT, function TEXT, EC_num TEXT, cycle TEXT);""")
conn.commit()

### Fill Pathways Table

In [128]:
c.execute("""INSERT INTO pathways VALUES('glycolysis', 'Glycolysis is the process of converting glucose into pyruvate and generating small amounts of ATP (energy) and NADH (reducing power).');""")
c.execute("""INSERT INTO pathways VALUES('TCA cycle', 'The citrate cycle (TCA cycle, Krebs cycle) is an important aerobic pathway for the final steps of the oxidation of carbohydrates and fatty acids.');""")
c.execute("""INSERT INTO pathways VALUES('pentose phosphate', 'The pentose phosphate pathway is a process of glucose turnover that produces NADPH as reducing equivalents and pentoses as essential parts of nucleotides.');""")
conn.commit()

In [129]:
organisms = ['Drosophila', 'Escherichia coli', 'Homo sapiens']

### Pick 4 enzymes from glycolysis, TCA cycle, and pentose phosphate

In [130]:
glycolysis_enzymes = ['Drosophila[ORGN] 5.1.3.3', 'Drosophila[ORGN] 2.7.1.1', 'Drosophila[ORGN] 3.1.3.9', 'Drosophila[ORGN] 5.3.1.9', 
                      'Escherichia coli[ORGN] 5.1.3.3', 'Escherichia coli[ORGN] 5.3.1.1', 'Escherichia coli[ORGN] 5.3.1.9', 'Escherichia coli[ORGN] 3.1.3.11',
                      'Homo sapiens[ORGN] 4.1.2.13', 'Homo sapiens[ORGN] 2.7.1.1', 'Homo sapiens[ORGN] 3.1.3.9', 'Homo sapiens[ORGN] 5.3.1.9']

TCA_enzymes = ['Drosophila[ORGN] 4.1.1.32', 'Drosophila[ORGN] 2.3.1.12', 'Drosophila[ORGN] 1.2.4.1', 'Drosophila[ORGN] 2.3.3.1', 
               'Escherichia coli[ORGN] 4.1.1.49', 'Escherichia coli[ORGN] 2.3.3.1', 'Escherichia coli[ORGN] 1.2.7.1', 'Escherichia coli[ORGN] 4.2.1.3',
               'Homo sapiens[ORGN] 4.1.1.32', 'Homo sapiens[ORGN] 2.3.1.12', 'Homo sapiens[ORGN] 1.2.4.1', 'Homo sapiens[ORGN] 2.3.3.1']

pentose_phsphate_enzymes = ['Drosophila[ORGN] 1.1.1.49', 'Drosophila[ORGN] 5.3.1.9', 'Drosophila[ORGN] 1.1.5.9', 'Drosophila[ORGN] 1.1.1.44', 
                            'Escherichia coli[ORGN] 1.1.1.49', 'Escherichia coli[ORGN] 2.2.1.1', 'Escherichia coli[ORGN] 3.1.1.31', 'Escherichia coli[ORGN] 5.3.1.6',
                            'Homo sapiens[ORGN] 3.1.1.17', 'Homo sapiens[ORGN] 3.1.1.31', 'Homo sapiens[ORGN] 5.1.3.1', 'Homo sapiens[ORGN] 1.1.1.49']

### Fill enzymes table

In [131]:
# enzyme name | description | EC number | cycle 
def enzyme_table(cycle_enzymes, cycle):
    enzymes = []
    EC_nums = []
    count = 0
    for enzyme in cycle_enzymes: 
        handle1 = Entrez.esearch(db = 'protein', term = enzyme, sort = 'relevance', idtype='acc')
        record = Entrez.read(handle1)['IdList']
        enzymes.append(record[0])
        EC_num = (enzyme.split()[-1])
        EC_nums.append(EC_num)
    for ID in enzymes: 
        handle2 = Entrez.efetch(db='protein', id=ID, rettype='gb', retmode='text')
        value = SeqIO.read(handle2, 'gb')
        c.execute(""" INSERT INTO enzymes (name, function, EC_num, cycle) VALUES(?,?,?,?)""", (value.name, value.description, EC_nums[count], cycle))
        count += 1

In [132]:
# If runtime error occurs, try running cell again
enzyme_table(glycolysis_enzymes, "Glycolysis")
conn.commit()

In [133]:
enzyme_table(TCA_enzymes, "TCA cycle")
conn.commit()

In [134]:
enzyme_table(pentose_phsphate_enzymes, "Pentose Phosphate")
conn.commit()

### Fill gene table 

In [135]:
# id INT PRIMARY KEY ASC, name TEXT, description TEXT, organism TEXT, nucleotide_seq TEXT, chromosome TEXT, start INT, end INT, strand VARCHAR(1), translated_seq TEXT
genes = []
def gene_table(cycle_enzymes, cycle):
    for gene in cycle_enzymes:
        ECnumber = gene.split("]")[1][1:]
        handle1 = Entrez.esearch(db = 'gene', term = ECnumber+"[EC]", sort = 'relevance', idtype='acc')
        record = Entrez.read(handle1)
        geneID = record['IdList']
        #print(geneID)
        for ID in geneID:
            handle2 = Entrez.efetch(db="gene", id=ID, rettype="gb", retmode="text")
            test = handle2.read()
            genes.append(test)
            print(test)
            #print(handle2.readline())
            #genes.append(handle2.readline())
            
                   
        #print(record)
        #print(gene)
        #print(ECnumber)
gene_table(glycolysis_enzymes, "Glycolysis")
print(genes)



1. galm
Official Symbol: galm and Name: galactose mutarotase [Danio rerio (zebrafish)]
Other Aliases: zgc:92501
Other Designations: aldose 1-epimerase
Chromosome: 13
Annotation: Chromosome 13 NC_007124.7 (8306407..8333891)
ID: 436646



1. GALM
Interim Symbol: GALM and Name: galactose mutarotase [Gallus gallus (chicken)]
Other Designations: aldose 1-epimerase; galactose mutarotase (aldose 1-epimerase)
Chromosome: 3
Annotation: Chromosome 3 NC_006090.5 (16619750..16634132)
ID: 426268



1. GALM
Official Symbol: GALM and Name: galactose mutarotase [Bos taurus (cattle)]
Other Designations: aldose 1-epimerase
Chromosome: 11
Annotation: Chromosome 11 NC_037338.1 (21021100..21075769)
ID: 616676



1. galm
Interim Symbol: galm and Name: galactose mutarotase (aldose 1-epimerase) [Xenopus tropicalis (tropical clawed frog)]
Other Designations: aldose 1-epimerase
Chromosome: 5
Annotation: Chromosome 5 NC_030681.1 (51799331..51823365)
ID: 549778



1. LOC100272618
aldose 1-epimerase [Zea mays]
Ot


1. HK2
Official Symbol: HK2 and Name: hexokinase 2 [Equus caballus (horse)]
Other Aliases: HKII
Other Designations: hexokinase-2; HK II; hexokinase II; hexokinase type II
Chromosome: 15
Annotation: Chromosome 15 NC_009158.3 (29191446..29249185, complement)
ID: 100009677



1. HK3
Official Symbol: HK3 and Name: hexokinase 3 [Bos taurus (cattle)]
Other Designations: hexokinase-3; hexokinase 3 (white cell)
Chromosome: 7
Annotation: Chromosome 7 NC_037334.1 (38332935..38352877, complement)
ID: 510616



1. hk2
Interim Symbol: hk2 and Name: hexokinase 2 [Xenopus tropicalis (tropical clawed frog)]
Other Designations: hexokinase-3
Chromosome: 3
Annotation: Chromosome 3 NC_030679.1 (37069333..37103646)
ID: 100145699



1. HK2
Official Symbol: HK2 and Name: hexokinase 2 [Pan troglodytes (chimpanzee)]
Other Designations: hexokinase-2
Chromosome: 2A
Annotation: Chromosome 2A NC_036880.1 (74984133..75044954)
ID: 741291



1. HK1
Official Symbol: HK1 and Name: hexokinase 1 [Pan troglodytes (chimpa


1. G6PC
Official Symbol: G6PC and Name: glucose-6-phosphatase catalytic subunit [Pan troglodytes (chimpanzee)]
Other Designations: glucose-6-phosphatase
Chromosome: 17
Annotation: Chromosome 17 NC_036896.1 (13878030..13890635, complement)
ID: 454703



1. g6pc3
glucose-6-phosphatase catalytic subunit 3 [Ictalurus punctatus (channel catfish)]
Other Designations: glucose-6-phosphatase 3
Chromosome: 13
Annotation: Chromosome 13 NC_030428.1 (23777379..23782255, complement)
ID: 100528077



1. G6pc
Official Symbol: G6pc and Name: glucose-6-phosphatase, catalytic [Mus musculus (house mouse)]
Other Aliases: AW107337, G6Pase, G6pt, Glc-6-Pase
Other Designations: glucose-6-phosphatase; Glc-6-Pase-alpha
Chromosome: 11; Location: 11 D
Annotation: Chromosome 11 NC_000077.6 (101367716..101377903)
ID: 14377



1. G6pc3
Official Symbol: G6pc3 and Name: glucose 6 phosphatase, catalytic, 3 [Mus musculus (house mouse)]
Other Aliases: 0710001K01Rik, AU019276, AU045429, AV128920, AW545836, UGRP
Other Des


1. GALM
Interim Symbol: GALM and Name: galactose mutarotase [Gallus gallus (chicken)]
Other Designations: aldose 1-epimerase; galactose mutarotase (aldose 1-epimerase)
Chromosome: 3
Annotation: Chromosome 3 NC_006090.5 (16619750..16634132)
ID: 426268



1. GALM
Official Symbol: GALM and Name: galactose mutarotase [Bos taurus (cattle)]
Other Designations: aldose 1-epimerase
Chromosome: 11
Annotation: Chromosome 11 NC_037338.1 (21021100..21075769)
ID: 616676



1. galm
Interim Symbol: galm and Name: galactose mutarotase (aldose 1-epimerase) [Xenopus tropicalis (tropical clawed frog)]
Other Designations: aldose 1-epimerase
Chromosome: 5
Annotation: Chromosome 5 NC_030681.1 (51799331..51823365)
ID: 549778



1. LOC100272618
aldose 1-epimerase [Zea mays]
Other Aliases: ZEAMMB73_Zm00001d034824, GRMZM5G898915
Other Designations: aldose 1-epimerase; Galactose mutarotase-like superfamily protein
Chromosome: 1
Annotation: Chromosome 1 NC_024459.2 (303002665..303005856, complement)
ID: 100272618


1. Dmoj\GI23383
Official Symbol: Dmoj\GI23383 and Name: GI23383 gene product from transcript GI23383-RA [Drosophila mojavensis]
Other Aliases: Dmoj_GI23383, GI23383, dmoj_GLEANR_8521
Other Designations: GI23383 gene product from transcript GI23383-RA; Dmoj\GI23383-PA; Dmoj\GI23383-PB; GI23383-PA; GI23383-PB; uncharacterized protein
Chromosome: Unknown
ID: 6572782



1. Dere\GG11962
Official Symbol: Dere\GG11962 and Name: GG11962 gene product from transcript GG11962-RA [Drosophila erecta]
Other Aliases: Dere_GG11962, GG11962, dere_GLEANR_12016
Other Designations: GG11962 gene product from transcript GG11962-RA; Dere\GG11962-PA; Dere\GG11962-PB; GG11962-PA; GG11962-PB; uncharacterized protein
Chromosome: Unknown
ID: 6554221



1. Dana\GF16201
Official Symbol: Dana\GF16201 and Name: GF16201 gene product from transcript GF16201-RA [Drosophila ananassae]
Other Aliases: Dana_GF16201, GF16201, dana_GLEANR_17472
Other Designations: GF16201 gene product from transcript GF16201-RA; Dana\GF16201


1. pgi
glucosephosphate isomerase [Escherichia coli str. K-12 substr. MG1655]
Other Aliases: b4025, ECK4017, JW3985
Annotation:  NC_000913.3 (4233758..4235407)
ID: 948535



1. PGI1
glucose-6-phosphate isomerase [Saccharomyces cerevisiae S288C]
Other Aliases: YBR196C, CDC30
Other Designations: glucose-6-phosphate isomerase
Chromosome: II
Annotation: Chromosome II NC_001134.8 (612236..613900, complement)
ID: 852495



1. GPI
glucose-6-phosphate isomerase [Sus scrofa (pig)]
Other Aliases: PGI
Other Designations: glucose-6-phosphate isomerase; AMF; NLK; PHI; autocrine motility factor; glucose phosphate isomerase; muscle phosphohexose isomerase; neuroleukin; phosphoglucose isomerase; phosphohexose isomerase
Chromosome: 6; Location: 6q12
Annotation: Chromosome 6 NC_010448.4 (44038374..44069331)
ID: 397602



1. gpia
Official Symbol: gpia and Name: glucose-6-phosphate isomerase a [Danio rerio (zebrafish)]
Other Aliases: PGI, gpi, pgi-1, wu:fc02c05, zgc:86919
Other Designations: glucose-6-ph


1. Aldoa
Official Symbol: Aldoa and Name: aldolase, fructose-bisphosphate A [Rattus norvegicus (Norway rat)]
Other Aliases: Aldo1, RNALDOG5
Other Designations: fructose-bisphosphate aldolase A; aldolase A, fructose-bisphosphate; muscle-type aldolase
Chromosome: 1; Location: 1q36
Annotation: Chromosome 1 NC_005100.4 (198228387..198233988, complement)
ID: 24189



1. ALDOA
Official Symbol: ALDOA and Name: aldolase, fructose-bisphosphate A [Homo sapiens (human)]
Other Aliases: ALDA, GSD12, HEL-S-87p
Other Designations: fructose-bisphosphate aldolase A; aldolase A, fructose-bisphosphate; epididymis secretory sperm binding protein Li 87p; fructose-1,6-bisphosphate triosephosphate-lyase; lung cancer antigen NY-LU-1; muscle-type aldolase
Chromosome: 16; Location: 16p11.2
Annotation: Chromosome 16 NC_000016.10 (30053090..30070420)
MIM: 103850
ID: 226



1. ALDOA
Official Symbol: ALDOA and Name: aldolase, fructose-bisphosphate A [Bos taurus (cattle)]
Other Designations: fructose-bisphosphate a


1. HK3
Official Symbol: HK3 and Name: hexokinase 3 [Pan troglodytes (chimpanzee)]
Other Designations: hexokinase-3; hexokinase 3 (white cell)
Chromosome: 5
Annotation: Chromosome 5 NC_036884.1 (154984211..155004870, complement)
ID: 462298



1. HK1
Official Symbol: HK1 and Name: hexokinase 1 [Canis lupus familiaris (dog)]
Other Designations: hexokinase-1
Chromosome: 4
Annotation: Chromosome 4 NC_006586.3 (20396761..20516889)
ID: 479234



1. Hkdc1
Official Symbol: Hkdc1 and Name: hexokinase domain containing 1 [Mus musculus (house mouse)]
Other Aliases: BC016235
Other Designations: putative hexokinase HKDC1; hexokinase domain-containing protein 1
Chromosome: 10; Location: 10 B4
Annotation: Chromosome 10 NC_000076.6 (62383137..62422464, complement)
ID: 216019



1. Hk2
Official Symbol: Hk2 and Name: hexokinase 2 [Mus musculus (house mouse)]
Other Aliases: AI642394, HKII
Other Designations: hexokinase-2; HK II; hexokinase type II
Chromosome: 6; Location: 6 35.94 cM
Annotation: Chromosom


1. G6PC2
Official Symbol: G6PC2 and Name: glucose-6-phosphatase catalytic subunit 2 [Pan troglodytes (chimpanzee)]
Other Designations: glucose-6-phosphatase 2; glucose-6-phosphatase, catalytic, 2
Chromosome: 2B
Annotation: Chromosome 2B NC_036881.1 (55464068..55472840)
ID: 741431



1. G6PC
glucose-6-phosphatase catalytic subunit [Felis catus (domestic cat)]
Other Aliases: G-6-Pase, G6Pase
Other Designations: glucose-6-phosphatase
Chromosome: E1
Annotation: Chromosome E1 NC_018736.3 (43389507..43398311)
ID: 723970



1. G6PC3
Official Symbol: G6PC3 and Name: glucose-6-phosphatase catalytic subunit 3 [Canis lupus familiaris (dog)]
Other Designations: glucose-6-phosphatase 3
Chromosome: 9
Annotation: Chromosome 9 NC_006591.3 (19299619..19304038, complement)
ID: 490942



1. G6PC3
Official Symbol: G6PC3 and Name: glucose-6-phosphatase catalytic subunit 3 [Pan troglodytes (chimpanzee)]
Other Designations: glucose-6-phosphatase 3; glucose 6 phosphatase, catalytic, 3
Chromosome: 17
Annotati


1. GPI
glucose-6-phosphate isomerase [Sus scrofa (pig)]
Other Aliases: PGI
Other Designations: glucose-6-phosphate isomerase; AMF; NLK; PHI; autocrine motility factor; glucose phosphate isomerase; muscle phosphohexose isomerase; neuroleukin; phosphoglucose isomerase; phosphohexose isomerase
Chromosome: 6; Location: 6q12
Annotation: Chromosome 6 NC_010448.4 (44038374..44069331)
ID: 397602



1. gpia
Official Symbol: gpia and Name: glucose-6-phosphate isomerase a [Danio rerio (zebrafish)]
Other Aliases: PGI, gpi, pgi-1, wu:fc02c05, zgc:86919
Other Designations: glucose-6-phosphate isomerase a; glucose phosphate isomerase a; pgi1
Chromosome: 25
Annotation: Chromosome 25 NC_007136.7 (35913678..35938996)
ID: 246094



1. PF_RS00995
glucose-6-phosphate isomerase [Pyrococcus furiosus DSM 3638]
Other Aliases: PF_RS00995, PF0196
Annotation:  NC_003413.1 (208695..209264)
ID: 1468028


['\n1. galm\nOfficial Symbol: galm and Name: galactose mutarotase [Danio rerio (zebrafish)]\nOther Aliases: zgc

In [139]:
name = []
desc = []
chromo = []
start = []
end = []
for gene in genes:
    aggregate = gene.splitlines()
    if aggregate:
        i = 0
        for line in aggregate:
            test = line.split()
            if test:
                if i == 0 and test[1].upper().lower() not in name:
                    name.append(test[1].upper().lower())
                if i == 1:
                    desc.extend(line.splitlines())
                if 'Annotation:' in test:
                    chromo.append(test[2])
                    for word in test:
                        if '..' in word:
                            tempPos = word[1:-1].split("..")
                            start.append(tempPos[0])
                            end.append(tempPos[1])
                i += 1;
        i += 1;
#print(name)
#print(desc)
        

In [140]:
i = 0
while i < len(name):
    c.execute(""" INSERT INTO genes (name, description, chromosome, start, end) VALUES(?,?,?,?,?)""", (name[i], desc[i], chromo[i], start[i], end[i]))
    i += 1
c.execute("""SELECT * FROM genes""")
print(c.fetchmany(100))

[('galm', 'Official Symbol: galm and Name: galactose mutarotase [Danio rerio (zebrafish)]', '13', '8306407', '8333891'), ('loc100272618', 'Interim Symbol: GALM and Name: galactose mutarotase [Gallus gallus (chicken)]', '3', '16619750', '16634132'), ('gal10', 'Official Symbol: GALM and Name: galactose mutarotase [Bos taurus (cattle)]', '11', '21021100', '21075769'), ('cg32444', 'Interim Symbol: galm and Name: galactose mutarotase (aldose 1-epimerase) [Xenopus tropicalis (tropical clawed frog)]', '5', '51799331', '51823365'), ('cg10467', 'aldose 1-epimerase [Zea mays]', '1', '303002665', '303005856'), ('cg10996', 'Official Symbol: GALM and Name: galactose mutarotase [Homo sapiens (human)]', '2', '38665910', '38734767'), ('cg4988', 'Official Symbol: Galm and Name: galactose mutarotase [Mus musculus (house mouse)]', '17', '80127457', '80185032'), ('cg32445', 'Official Symbol: Galm and Name: galactose mutarotase [Rattus norvegicus (Norway rat)]', '6', '2808988', '2860742'), ('exig_rs02130',

### Fill Associative Table

In [146]:
c.execute("""DROP TABLE enzyme_pathway""")
c.execute("""CREATE TABLE enzyme_pathway AS SELECT cycle, name, EC_num FROM enzymes""")
conn.commit()

### Print Enzymes Table

In [147]:
c.execute("""SELECT * FROM enzymes""")
print(c.fetchmany(36))

[('NP_730671', 'uncharacterized protein Dmel_CG32445 [Drosophila melanogaster]', '5.1.3.3', 'Glycolysis'), ('XP_002055038', 'uncharacterized protein Dvir_GJ19157 [Drosophila virilis]', '2.7.1.1', 'Glycolysis'), ('NP_001097063', 'Glucose-6-Phosphatase [Drosophila melanogaster]', '3.1.3.9', 'Glycolysis'), ('XP_002048914', 'uncharacterized protein Dvir_GJ21054 [Drosophila virilis]', '5.3.1.9', 'Glycolysis'), ('RIC81143', 'galactose-1-epimerase, partial [Escherichia coli]', '5.1.3.3', 'Glycolysis'), ('AXZ41087', 'triose-phosphate isomerase [Escherichia coli]', '5.3.1.1', 'Glycolysis'), ('AXZ40953', 'glucose-6-phosphate isomerase [Escherichia coli]', '5.3.1.9', 'Glycolysis'), ('RIC83436', 'class II fructose-bisphosphatase, partial [Escherichia coli]', '3.1.3.11', 'Glycolysis'), ('AAA51697', 'fructose 1,6-diphosphate aldolase A (EC 4.1.2.13), partial [Homo sapiens]', '4.1.2.13', 'Glycolysis'), ('NP_002106', 'hexokinase-3 [Homo sapiens]', '2.7.1.1', 'Glycolysis'), ('NP_612396', 'glucose-6-pho

### Print Pathways Table

In [148]:
c.execute("""SELECT * FROM pathways""")
print(c.fetchmany(3))

[('glycolysis', 'Glycolysis is the process of converting glucose into pyruvate and generating small amounts of ATP (energy) and NADH (reducing power).'), ('TCA cycle', 'The citrate cycle (TCA cycle, Krebs cycle) is an important aerobic pathway for the final steps of the oxidation of carbohydrates and fatty acids.'), ('pentose phosphate', 'The pentose phosphate pathway is a process of glucose turnover that produces NADPH as reducing equivalents and pentoses as essential parts of nucleotides.')]


### Print Associative Table
Each enzyme we chose is unique to its pathway. Enzymes with the same EC number come from the same pathway. 
There is an order to enzymes in pathways. For glycolysis, step 1 is hexokinase, step 2 is phosphoglucose isomerase, step 3 is phosphofructokinase, step 4 is aldolase. 

In [149]:
c.execute("""SELECT * FROM enzyme_pathway""")
print(c.fetchmany(36))

[('Glycolysis', 'NP_730671', '5.1.3.3'), ('Glycolysis', 'XP_002055038', '2.7.1.1'), ('Glycolysis', 'NP_001097063', '3.1.3.9'), ('Glycolysis', 'XP_002048914', '5.3.1.9'), ('Glycolysis', 'RIC81143', '5.1.3.3'), ('Glycolysis', 'AXZ41087', '5.3.1.1'), ('Glycolysis', 'AXZ40953', '5.3.1.9'), ('Glycolysis', 'RIC83436', '3.1.3.11'), ('Glycolysis', 'AAA51697', '4.1.2.13'), ('Glycolysis', 'NP_002106', '2.7.1.1'), ('Glycolysis', 'NP_612396', '3.1.3.9'), ('Glycolysis', 'AAB36062', '5.3.1.9'), ('TCA cycle', 'XP_002049346', '4.1.1.32'), ('TCA cycle', 'XP_015028645', '2.3.1.12'), ('TCA cycle', 'XP_002052798', '1.2.4.1'), ('TCA cycle', 'XP_015025715', '2.3.3.1'), ('TCA cycle', 'AXZ41729', '4.1.1.49'), ('TCA cycle', 'AXZ44819', '2.3.3.1'), ('TCA cycle', 'SVF52481', '1.2.7.1'), ('TCA cycle', 'AXZ43524', '4.2.1.3'), ('TCA cycle', 'PCKGM_HUMAN', '4.1.1.32'), ('TCA cycle', 'NP_001922', '2.3.1.12'), ('TCA cycle', 'AAA36533', '1.2.4.1'), ('TCA cycle', 'NP_004068', '2.3.3.1'), ('Pentose Phosphate', 'XP_002048

In [150]:
c.execute("""SELECT * FROM enzyme_pathway WHERE EC_num = '2.3.3.1'""")
print(c.fetchmany(36))

[('TCA cycle', 'XP_015025715', '2.3.3.1'), ('TCA cycle', 'AXZ44819', '2.3.3.1'), ('TCA cycle', 'NP_004068', '2.3.3.1')]


## Scratch

In [118]:
from Bio.KEGG import REST
from Bio.KEGG import Enzyme
request = REST.kegg_get("dme:5.1.3.3")
open("dme_5.1.3.3.txt", "w").write(request.read())
records = Enzyme.parse(open("dme_5.1.3.3.txt"))
record = list(records)[0]
print(record.classname)
record.entry

HTTPError: HTTP Error 404: Not Found

In [ ]:
from Bio.KEGG import REST

dme_pathways = REST.kegg_list("pathway", "dme").read()
dme_pathways
# Filter all human pathways for repair pathways
repair_pathways = []
for line in dme_pathways.rstrip().split("\n"):
    entry, description = line.split("\t")
    if "metabolism" in description:
        repair_pathways.append(entry)
        print(entry)
        c.execute("""SELECT * from pathways""")
       # c.execute("INSERT INTO pathways (pathway) VALUES (%s);", 'hi')
print(repair_pathways)
# Get the genes for pathways and add them to a list
repair_genes = [] 
for pathway in repair_pathways:
    pathway_file = REST.kegg_get(pathway).read()  # query and read each pathway

    # iterate through each KEGG pathway file, keeping track of which section
    # of the file we're in, only read the gene in each pathway
    current_section = None
    for line in pathway_file.rstrip().split("\n"):
        #print(line)
        section = line[10:].strip()  # section names are within 12 columns
        print(section)
        if not section == "":
            current_section = section
        
        if current_section == "GENE":
            gene_identifiers, gene_description = line[:2].split("; ")
            gene_id, gene_symbol = gene_identifiers.split()

            if not gene_symbol in repair_genes:
                repair_genes.append(gene_symbol)

#print("There are %d repair pathways and %d repair genes. The genes are:" % \
#(len(repair_pathways), len(repair_genes)))
#print(", ".join(repair_genes))

In [152]:
conn.close()

ProgrammingError: Cannot operate on a closed database.

$ sqlite3 <br>
.tables <br>
enzymes genes p pathways" <br>
SELECT * FROM ENZYMES